# Notebook example using Kaggle GPU

In [1]:
import numpy as np
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms


from tqdm import tqdm
import PIL.Image as Image

# Load the Drive helper and mount
from google.colab import drive
import os

# This will prompt for authorization.
drive.mount( 'Drive')
persistent_storage = 'Drive/My Drive/recvis21_a3/'
os.makedirs(persistent_storage, exist_ok=True)

if not os.path.isdir('Drive/My Drive/recvis21_a3/experiments'):
    os.makedirs('Drive/My Drive/recvis21_a3/experiments')
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Drive already mounted at Drive; to attempt to forcibly remount, call drive.mount("Drive", force_remount=True).


In [2]:
#Change them / create some others ?
batch_size = 8
epochs = 20
seed=0
torch.manual_seed(seed)

# Data Loading & Model creation

In [3]:
data_transforms = {
    "train": transforms.Compose(
        [
            transforms.Resize((299, 299)),
            # transforms.transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(0.3),
            transforms.RandomPerspective(distortion_scale=0.4, p=0.5),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    ),
    "val": transforms.Compose(
        [
            transforms.Resize((299, 299)),
            # transforms.transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(0.3),
            transforms.RandomPerspective(distortion_scale=0.4, p=0.5),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    ),
    "test": transforms.Compose(
        [
            transforms.Resize((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
    ),
}


train_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder('Drive/My Drive/recvis21_a3/299_cropped_bird_dataset/train_images', transform=data_transforms["train"]),
    batch_size=batch_size, shuffle=True, num_workers=1)

val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder('Drive/My Drive/recvis21_a3/299_cropped_bird_dataset/val_images',transform=data_transforms["val"]),
    batch_size=batch_size, shuffle=True, num_workers=1)

test_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder('Drive/My Drive/recvis21_a3/299_cropped_bird_dataset/test_images',transform=data_transforms["test"]),
    batch_size=1, shuffle=False, num_workers=1)

In [4]:
from torchvision import models

class Net(nn.Module):
    def __init__(self, num_classes=20):
        super(Net, self).__init__()

        #self.res = models.resnet152(pretrained=True)
        self.inc = models.inception_v3(pretrained=True)

        for param in self.inc.parameters():
            param.requires_grad = True
        self.inc.aux_logits = False
        num_features = self.inc.fc.in_features
        self.inc.fc = nn.Linear(num_features, 512)

        #for param in self.res.conv1.parameters():
        #    param.requires_grad = True
        #for param in self.res.bn1.parameters():
        #    param.requires_grad = True
        #for param in self.res.layer1.parameters():
        #    param.requires_grad = True
        #for param in self.res.layer2.parameters():
        #    param.requires_grad = True
        #for param in self.res.layer3.parameters():
        #    param.requires_grad = True

        #self.res.avgpool = nn.AvgPool2d(10)
        #num_features2 = self.res.fc.in_features
        #self.res.fc = nn.Linear(num_features2, 512)

        self.eff = models.efficientnet_b7(pretrained=True)

        for param in self.eff.parameters():
            param.requires_grad = True

        num_features2 = self.eff.classifier[1].in_features

        self.eff.classifier = nn.Sequential(
            nn.Dropout(p=0.4, inplace=True),
            nn.Linear(num_features2, 512),
        )

        lin3 = nn.Linear(1024, num_classes)
        self.fc = lin3

    def forward(self, input):
        x1 = self.eff(input)
        x2 = self.inc(input)
        x = torch.cat((x1, x2), 1)
        return self.fc(x)

    
model = Net()
model.to(device)

optimizer = optim.SGD(
    model.parameters(),
    lr=.001,
    momentum=0.9,
)
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
criterion = torch.nn.CrossEntropyLoss()

# Train

In [5]:
def train(model, epoch):
    model.train()
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        #forward
        preds = model(data)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        if batch_idx % 10 == 0:
            print('[{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))


def validation(model):
    model.eval()
    validation_loss = 0
    correct = 0
    with torch.no_grad():
        for data, labels in val_loader:
            data, labels = data.to(device), labels.to(device)
            preds = model(data)
            # sum up batch loss
            validation_loss += criterion(preds, labels).data.item()
            m = nn.Softmax(dim=1)
            probs = m(preds)
            preds_classes = probs.max(1, keepdim=True)[1]
            correct += preds_classes.eq(labels.data.view_as(preds_classes)).sum()
        validation_loss /= len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))

In [ ]:
for epoch in range(1, epochs + 1):
    print("################################################# EPOCH", epoch)
    train(model, epoch)
    preds = validation(model)
    model_file = 'Drive/My Drive/recvis21_a3/experiments' + '/model_' + str(epoch) + '.pth'
    torch.save(model.state_dict(), model_file)

################################################# EPOCH 1
[0/1092 (0%)]	Loss: 3.006087


In [ ]:
break

# Test

In [ ]:
preds = np.array([])
model.eval()
with torch.no_grad():
    for i, (data, labels) in tqdm(enumerate(test_loader, 0)):
        data, labels = data.to(device), labels.to(device)
        output1 = model(data)
        sm = nn.Softmax(dim=1)(output1)
        pred = sm.max(1, keepdim=True)[1]    
        preds = np.hstack((preds, torch.squeeze(pred).cpu().numpy()))

In [ ]:
f = open("Drive/My Drive/recvis21_a3/299_cropped_submission.csv", "w")
f.write("Id,Category\n")
for (n,_),p in zip(test_loader.dataset.samples,preds):
    f.write("{},{}\n".format(n.split('/')[-1].split('.')[0], int(p)))
f.close()